# 대회 전략 구현하기

### ✨실습 개요<br>

1) 실습 목적 <br>
  이번 실습에서는 대회에서 사용되는 전략들을 구현해봅니다. <br>
  자연어 데이터를 증강하는 3가지 방법과, 후처리 기법인 앙상블 그리고 모델을 MLM으로 추가학습하는 코드를 구현합니다.   <br>


 2) 수강 목표
  - 자연어 데이터를 증강할 수 있다.
  - 학습 시 k-fold로 데이터를 나눌 수 있다.
  - Ensemble을 통해서 점수를 올려볼 수 있다.
  - MLM Task를 구현하고 DAPT & TAPT를 실행할 수 있다.

#### 실습 목차
1. 데이터 증강기법 살펴보기
    * 1-1. EDA 사용해보기
    * 1-2. AEDA 사용해보기
    * 1-3. Back Translation(feat. Google Translate)
2. K-fold & Ensemble
  * 2-1. K-fold 로 데이터 나누기
  * 2-2. Ensemble로 성능 올리기
3. DAPT & TAPT 구현하기
  * 3-1. 학습에 쓰일 데이터셋 준비하기
  * 3-2. 데이터셋 구성 및 마스킹하기
  * 3-3. MLM 학습하기
  


In [ ]:
%pip install koeda
%pip install googletrans==4.0.0-rc1
%pip install git+https://github.com/kakaobrain/pororo.git

In [ ]:
!mkdir ./dataset/fake_news
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1gucFY-P9a1TzdV8Xb-OwVD4TpPy4iyJX' -O ./dataset/fake_news/train.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1J05RaqfknDzTObofL7OyiSmT0B8rX0gZ' -O ./dataset/fake_news/test.csv

In [ ]:
import pandas as pd

train_df = pd.read_csv("./dataset/fake_news/train.csv",index_col=0)
test_df = pd.read_csv("./dataset/fake_news/test.csv",index_col=0)

train_df.head(3)

In [ ]:
from datasets import load_dataset

news_dump = load_dataset("krenerd/korean-newstext-dump", split='train')

In [ ]:
news_dump["text"][:4]

# KoEDA

영어용으로 구현된 Easy data augmentation 과 An Easier Data Augmentation 프로젝트를 한글용으로 재구성한 프로젝트입니다.

EasyDataAugmentation 파라미터
 - `p = (alpha_sr, alpha_ri, alpha_rs, prob_rd)` 은 <br> SR, RI , RS , RD 에 대한 각각의 변환을 어느정도 비율로 할 것인지 결정
 - Synonym Replacement __(SR)__ : 유의어 교체
 - Random Insertion __(RI)__ : 임의 단어 삽입
 - Random Swap __(RS)__ : 두 단어 위치 변경
 - Random Deletion __(RD)__ : 임의 단어 삭제

 morpheme_analyzer 는 사용될 형태소 분석기를 지정하는 파라미터로, <br> ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"] 중 하나를 선택할 수 있다.  <br>__(단, 일부는 설치 필요하며, 각각 형태소를 나누는 기준이 다르다. )__

In [ ]:
# 예시 데이터 고정
ex_data = news_dump[1]['text']
ex_data

## Easy Data Augmentation

In [ ]:
from koeda import EasyDataAugmentation

def augment_text_data_with_EDA(text,repetition):
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(morpheme_analyzer="Okt")

    result = eda(text,p=(0.5, 0.5, 0.5, 0.5), repetition=repetition)

    # 증강 결과 출력
    print("원문: " , text)
    print("--"*100)
    for i in range(repetition):
        print(f"증강문{i+1}: ", result[i])
    # return result

In [ ]:
augment_text_data_with_EDA(ex_data,3)

## AnEasierDataAugmentation

AnEasierDataAugmentation 파라미터
 - `p = punc_ratio` 는 punctuations 로의 변환을 어느정도 비율로 할 것인지 결정
 - punctuations 는 ['.', ',', '!', '?', ';', ':'] 로 입력
 - morpheme_analyzer 는 사용될 형태소분석기를 지정하는 파라미터로, <br> ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"] 중 하나를 선택할 수 있다.  <br>__(단, 일부는 설치 필요하며, 각각 형태소를 나누는 기준이 다르다. )__

In [ ]:
from koeda import AEasierDataAugmentation
def augment_text_data_with_AEDA(text, repetition):
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(morpheme_analyzer="Okt", punctuations=[".", ",", "!", "?", ";", ":"])

    result = aeda(text, p=0.3, repetition=repetition)

    # 증강 결과 출력
    print("원문: " , text)
    print("--"*100)
    for i in range(repetition):
        print(f"증강문{i+1}: ", result[i])
    # return result

In [ ]:
augment_text_data_with_AEDA(ex_data,2)

# BackTranslation

In [ ]:
from googletrans import Translator


def augment_text_data_with_BT(text, repetition):
    """입력된 문장에 대해서 BT를 통해 데이터 증강"""
    # Translator 객체 생성
    translator = Translator()
    result = []

    # 번역 실행 (한국어 > 영어 > 한국어)
    for i in range(repetition):
        translated = translator.translate(text, src='ko', dest='en')
        re_translated = translator.translate(translated.text, src='en', dest='ko')
        result.append(re_translated.text)

    # 번역 결과 출력
    print("원문: " , text)
    print("--"*100)
    for i in range(repetition):
        print(f"증강문{i+1}: ", result[i])
    # return result

In [ ]:
augment_text_data_with_BT(ex_data, 1)

In [ ]:
# from pororo import Pororo

# def augment_text_data_with_pororo_BT(text, repetition):
#     """입력된 문장에 대해 Pororo 모델을 이용하여 BT로 데이터 증강"""
#     # Pororo 번역 모델 초기화 (ko -> en, en -> ko)
#     translator_ko_en = Pororo(task="translation", lang="ko", tgt="en")
#     translator_en_ko = Pororo(task="translation", lang="en", tgt="ko")
    
#     result = []

#     # 번역 실행 (한국어 > 영어 > 한국어)
#     for i in range(repetition):
#         translated = translator_ko_en(text)
#         re_translated = translator_en_ko(translated)
#         result.append(re_translated)

#     # 번역 결과 출력
#     print("원문: " , text)
#     print("--" * 100)
#     for i in range(repetition):
#         print(f"증강문 {i+1}: ", result[i])
    
#     return result


# DAPT, TAPT

In [ ]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer , AutoModelForMaskedLM
from torch.utils.data import Dataset, DataLoader, RandomSampler

import pytorch_lightning as pl
import torch

In [ ]:
ex_news_dump = news_dump['text'][:50000]

In [ ]:
class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer, data, block_size):
        concated_ls = []
        # 제목 + 본문으로 합치기
        for i in range(1,len(data)):
            concated_ls.append(data[i-1] + data[i])

        batch_encoding = tokenizer(concated_ls, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]
        self.examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in self.examples]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [ ]:
def prepare_dataset_for_pretraining(tokenizer,train_input):
    train_dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        data=train_input,
        block_size=512,
    )
    # set mlm task
    # DataCollatorForSOP로 변경시 SOP 사용 가능 (DataCollatorForLanguageModeling)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15 # 0.3
    )
    eval_dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        data=train_input[:20],
        block_size=512,
    )

    return train_dataset, data_collator, eval_dataset

In [ ]:
def set_trainer_for_pretraining(
        model,
        data_collator,
        dataset,
        eval_dataset,
        epoch = 10,
        batch_size = 16,
        accumalation_step = 1,):
     # set training args
    training_args = TrainingArguments(
        report_to = 'tensorboard',
        output_dir='./',
        overwrite_output_dir=True,
        num_train_epochs=epoch,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=accumalation_step,
        evaluation_strategy = 'steps',
        eval_steps=150,
        save_steps=150,
        save_total_limit=1,
        fp16=True,
        load_best_model_at_end=True,
        seed=42,
    )

    # set Trainer class for pre-training
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
        eval_dataset=eval_dataset,
    )

    return trainer

In [ ]:
def pretrain():
    """MLM task 기반 사전학습 진행"""
    # fix a seed
    pl.seed_everything(seed=42, workers=False)

    # set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device:", device)

    # set model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForMaskedLM.from_pretrained("klue/bert-base")
    model.to(device)

    # set data
    train_dataset, data_collator, eval_dataset = prepare_dataset_for_pretraining(tokenizer, ex_news_dump)

    # set trainer
    trainer = set_trainer_for_pretraining(model,data_collator,train_dataset,eval_dataset)

    # train model
    print("--- Start train ---")
    trainer.train()
    print("--- Finish train ---")
    model.save_pretrained("./pretrained")

In [ ]:
pretrain()